In [ ]:
!pip install tf-nightly tensorflow-text tensorflow-addons tf-models-official scikit-learn tensorflow-gpu

In [2]:
import re
import pandas as pd
from google.colab import drive 
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow_addons as tfa

from official.nlp import optimization 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [3]:
drive.mount('/content/gdrive')

data_path = "/content/gdrive/MyDrive/Colab Notebooks/sentimaster_data/"
train_file = data_path + "train_complete.csv"
test_file = data_path + "test_text.txt"

random_state = 42
df = pd.read_csv(train_file)
df.head()

Mounted at /content/gdrive


,tweet,label
0,"""QT @user In the original draft of the 7th boo...",2
1,"""Ben Smith / Smith (concussion) remains out of...",1
2,Sorry bout the stream last night I crashed out...,1
3,Chase Headley's RBI double in the 8th inning o...,1
4,@user Alciato: Bee will invest 150 million in ...,2


In [4]:
def text_preprocessing_bert(text):

    # Remove @mentions
    text = re.sub(r'(@.*?)[\s]', ' ', text)
    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)
    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text


def get_bert_preprocessed_dataset(df):
    df['tweet'] = df['tweet'].map(text_preprocessing_bert)
    return df


tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(3, activation='softmax', name='classifier')(net)
  return tf.keras.Model(text_input, net)




classifier_model = build_classifier_model()
loss = tf.keras.losses.CategoricalCrossentropy()
metrics = [tfa.metrics.F1Score(num_classes=3, average='macro'),
            tf.keras.metrics.CategoricalAccuracy()]


bert_df = get_bert_preprocessed_dataset(df)


label_encoder = LabelBinarizer()
label_encoder.fit(df['label'].values)

X = bert_df['tweet'].values
y = label_encoder.transform(bert_df['label'].values)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, 
                                                    random_state=random_state)

train_tf_df = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
val_tf_df = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(32)



bert_model = hub.KerasLayer(tfhub_handle_encoder)
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)


epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(train_tf_df).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')


classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [5]:
history = classifier_model.fit(x=train_tf_df,
                               validation_data=val_tf_df,
                               epochs=epochs)

Epoch 1/5
1340/1340 [==============================] - 125s 89ms/step - loss: 1.0033 - f1_score: 0.4474 - categorical_accuracy: 0.5070 - val_loss: 0.7845 - val_f1_score: 0.5789 - val_categorical_accuracy: 0.6430
Epoch 2/5
1340/1340 [==============================] - 110s 82ms/step - loss: 0.7900 - f1_score: 0.5906 - categorical_accuracy: 0.6356 - val_loss: 0.7458 - val_f1_score: 0.6212 - val_categorical_accuracy: 0.6630
Epoch 3/5
1340/1340 [==============================] - 113s 84ms/step - loss: 0.7456 - f1_score: 0.6262 - categorical_accuracy: 0.6613 - val_loss: 0.7318 - val_f1_score: 0.6352 - val_categorical_accuracy: 0.6735
Epoch 4/5
1340/1340 [==============================] - 115s 86ms/step - loss: 0.7217 - f1_score: 0.6412 - categorical_accuracy: 0.6739 - val_loss: 0.7258 - val_f1_score: 0.6493 - val_categorical_accuracy: 0.6772
Epoch 5/5
1340/1340 [==============================] - 111s 83ms/step - loss: 0.7026 - f1_score: 0.6586 - categorical_accuracy: 0.6875 - val_loss: 0.722

In [6]:
classifier_model.save(data_path + "classifier.h5")

In [9]:
import numpy as np
with open(test_file) as f:
    challange_test_tweets = [
            text_preprocessing_bert(line.rstrip()) for line in f
        ]

In [14]:
predictions = classifier_model(tf.constant(challange_test_tweets))

In [20]:
with open(test_file) as f:
    original_tweets = [line.rstrip() for line in f]

csv_result = pd.DataFrame()
csv_result['tweet'] = original_tweets
csv_result['label'] = label_encoder.inverse_transform(predictions.numpy())

csv_result.head()

,tweet,label
0,@user @user what do these '1/2 naked pics' hav...,1
1,OH: “I had a blue penis while I was this” [pla...,1
2,"@user @user That's coming, but I think the vic...",1
3,I think I may be finally in with the in crowd ...,2
4,"@user Wow,first Hugo Chavez and now Fidel Cast...",2


In [22]:
csv_result.to_csv(data_path + 'model_predictions.csv', index=False)